In [1]:
import import_ipynb
from frame_slicer import save_frames
import image_process
import video_slicer as vs
import os

importing Jupyter notebook from image_process.ipynb
importing Jupyter notebook from video_slicer.ipynb


In [2]:
# 선택한 확장자만 가진 파일명 추출하는 func
def getfilenames(filenameslist:list, extension:str)->list:
    return [i for i in filenameslist if extension in i]

# 같은 이름의 mp4, json mapping
def zipper(videos_name_list:list, jsons_name_list:list):
    videos_name_list = getfilenames(videos_name_list, '.mp4')
    jsons_name_list = getfilenames(jsons_name_list, '.json')
    if len(videos_name_list) == len(jsons_name_list):
        return zip(sorted(videos_name_list), sorted(jsons_name_list))
    else:
        raise Exception('Missing file Detected')


# mp4, json parent folder로부터 image 추출하는 func
""" Data files format:
parent_folder_dir (ex. 2.Validation)
    ├ parent2_videos_folder_dir (CROWD)
	  ├─ videos_folder_dir (18)  <- 이 dir을 getImage에 넣어야함 1
		├─ videos (NIA_SL_FSXXXX_CROWDXX_F.mp4 ...)
	  ├─ videos_folder_dir (19)  <- 이 dir을 getImage에 넣어야함 2
		├─ videos (NIA_SL_FSXXXX_CROWDXX_F.mp4 ...)
      ...
    ├ parent_json_folder_dir (morpheme)
	  ├─ jsons_folder_dir (18)  <- 이 dir을 getImage에 넣어야함 1
		├─ jsons (NIA_SL_FSXXXX_CROWDXX_F_morpheme.json ...)
	  ├─ jsons_folder_dir (19)  <- 이 dir을 getImage에 넣어야함 2
		├─ jsons (NIA_SL_FSXXXX_CROWDXX_F_morpheme.json ...)
      ...
"""

""" Usage guide:
v = './test/parent_video/'
j = './test/parent_json/'
[i for i in os.walk(v)][0] <==> ├─ videos_folder_dir (18)
[i for i in os.walk(j)][0] <==> ├─ jsons_folder_dir (18)
"""

def getImage(videos_folder_dir:str, jsons_folder_dir:str):
    # [0] in info : direct connected file with folder
    videos_folder_info = [i for i in os.walk(video_folder_dir)][0]
    jsons_folder_info = [i for i in os.walk(jsons_folder_dir)][0]
    # videos_folder_dir = videos_folder_info[0]
    # jsons_folder_dir = jsons_folder_info[0]
    videos_name_list = videos_folder_info[2]
    jsons_name_list = jsons_folder_info[2]
    output1_video_folder_dir = os.path.join(videos_folder_dir, os.pardir, 'cutted_video')
    os.makedirs(output1_video_folder_dir, exist_ok=True)
    output2_video_folder_dir = os.path.join(videos_folder_dir, os.pardir, 'resized_video')
    os.makedirs(output2_video_folder_dir, exist_ok=True)

    for video_name, json_name in zipper(videos_name_list, jsons_name_list):
        inputvideo_dir = os.path.join(videos_folder_dir, video_name)
        # get json info
        json_dir = os.path.join(jsons_folder_dir, json_name)
        start, end, name = vs.extract_start_end_time(json_dir)
        # cutting video
        outputvideo1_dir = os.path.join(output1_video_folder_dir, video_name.split('.')[0]) + '_cutted.mp4'
        vs.cut_video(inputvideo_dir, outputvideo1_dir, start, end)
        # resizing video
        outputvideo2_dir = os.path.join(output2_video_folder_dir, video_name.split('.')[0]) + '_resized.mp4'
        vs.convert_and_resize_video(outputvideo1_dir, outputvideo2_dir, 405, 720)
        # video to img
        outputimgs_dir = os.path.join(videos_folder_dir, os.pardir, 'result_img', name)
        os.makedirs(outputimgs_dir, exist_ok=True)
        save_frames(outputvideo2_dir, outputimgs_dir)
        # remove the blurred images
        img = image_process.ImageProcess()
        img.removeBlurImage(outputimgs_dir, 55, print_img=False)

video_folder_dir = './test/parent_video/'
jsons_folder_dir = './test/parent_json/'

getImage(video_folder_dir, jsons_folder_dir)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


Frames saved to ./test/parent_video/../result_img/NIA_SL_FS0001_CROWD18_F.
score mean:  51.575685316781616
86 / 118 images deleted
